<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020SightVisit_Decision_tree_Random_forest_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle の タイタニック号沈没 データ `https://www.kaggle.com/c/titanic/data` から
ファイルを入手してください。

ただし，データの入手には kaggle のアカウントを作成する必要があります。
著作権の問題ですので，kaggle アカウントを作成しなければデータを入手できません。
アカウント作成はそれほど難しくありません。

コマンドラインとしては以下のとおりです
```bash
$ kaggle competitions download -c titanic
```

In [ ]:
from google.colab import files
uploaded = files.upload()  # kaggle から入手した `titanic.csv` をアップロードしてください 

In [ ]:
# アップロードしたファイルを確認します
!head titanic.csv
!wc titanic.csv


In [ ]:
import numpy as np
import os
import csv

In [ ]:
X, y = np.zeros((891,5), dtype=np.float), np.zeros((891,),dtype=np.float)

with open('titanic.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        id = int(row['PassengerId']) - 1
        y[id,] = int(row['Survived'])
        try:
            X[id, 0] = float(row['Age'])
        except:
            continue
        X[id,1] = 1 if row['Sex'] == 'male' else 0
        _class = row['Pclass']
        if _class == '1':
            X[id,2] = 1
        elif _class == '2':
            X[id,3] = 1
        else:
            X[id,4] = 1

In [ ]:
feature_names = ['age', 'sex', 'first_class', 'second_class', 'third_class']
print(X[:3])  # 確認のため最初の 3 行分のデータを表示

In [ ]:
from sklearn.model_selection import train_test_split
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# 判断木の実行
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', 
                                  max_depth=3,
                                  min_samples_leaf=5)
clf = clf.fit(X_train, y_train) # clf.fit() は訓練させることを意味します

In [ ]:
import pydotplus
import io

dot_data = io.StringIO() 
tree.export_graphviz(clf, out_file=dot_data, feature_names=['age','sex','1st_class','2nd_class','3rd_class']) 
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_png('titanic.png') 
from IPython.core.display import Image 
Image(filename='titanic.png')

In [ ]:
# 精度の表示
from sklearn import metrics
def measure_performance(X, y,
                        clf, 
                        show_accuracy=True, 
                        show_classification_report=True, 
                        show_confusion_matrix=True):
    y_pred=clf.predict(X)   
    if show_accuracy:
        print("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n")

    if show_classification_report:
        print("Classification report")
        print(metrics.classification_report(y,y_pred),"\n")
        
    if show_confusion_matrix:
        print("Confusion matrix")
        print(metrics.confusion_matrix(y,y_pred),"\n")
        
measure_performance(X_train,
                    y_train,
                    clf, 
                    show_classification_report=True, 
                    show_confusion_matrix=True)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("titanic")


dot_data = tree.export_graphviz(clf, 
                                out_file=None, 
                                feature_names=feature_names, 
                                class_names=['Survived', 'Not'],
                                filled=True, rounded=True, 
                                special_characters=True)

graph = graphviz.Source(dot_data) 
graph.render('titanic')                                                                                         

In [ ]:
files.download('titanic.pdf')

In [ ]:
# ランダムフォレストによる分析
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train,y_train)

measure_performance(X_train,
                    y_train,
                    clf, 
                    show_classification_report=True, 
                    show_confusion_matrix=True)